In [1]:
from pyhive import hive
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import sqlalchemy

from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from joblib import load
import joblib
from lightgbm.sklearn import LGBMClassifier,LGBMRegressor
from hdfs import InsecureClient
from sklearn.model_selection import  KFold
# Hive 연결
conn = hive.Connection(host='172.31.1.129', port=10000, 
                       username='hive', password='1234', auth='LDAP')
cursor = conn.cursor()

query = '''
select 
  pollution.ymd as ymd,
  pollution.hour as hour,
  pollution.spot as spot,
  pollution.pm10 as pm10,
  pollution.pm25 as pm25,
  pollution.o3 as o3,
  pollution.no2 as no2,
  pollution.co as co,
  pollution.so2 as so2
from pollution
'''
query2 = '''
select traffic.ymd as ymd,
traffic.hour as hour ,
traffic.spot as spot,
traffic.vol as vol
from traffic
'''

# 쿼리 실행
cursor.execute(query)

# 컬럼 정보 가져오기
columns = [desc[0] for desc in cursor.description]

# 결과 가져오기
data = cursor.fetchall()
cursor.execute(query2)
columns1 = [desc[0] for desc in cursor.description]
data1 = cursor.fetchall()

# 커서와 연결 닫기
conn.close()

# 결과를 데이터프레임으로 변환
air_df = pd.DataFrame(data, columns=columns)
tr_df = pd.DataFrame(data1, columns=columns1)
# 결과 출력



In [56]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS predict (
    gu STRING,
    ymd DATE,
    hour INT,
    traffic FLOAT
)
'''

In [55]:
conn = hive.Connection(host='172.31.1.129', port=10000, 
                       username='hive', password='1234', auth='LDAP')
cursor = conn.cursor()
al_query = '''
DROP TABLE default.predict'''
# 테이블 생성 쿼리 실행
cursor.execute(al_query)

# 커밋
conn.commit()

# 커넥션 닫기
conn.close()

In [57]:
conn = hive.Connection(host='172.31.1.129', port=10000, 
                       username='hive', password='1234', auth='LDAP')

# ALTER TABLE default.predict
# CHANGE COLUMN ymd ymd DATE
cursor = conn.cursor()
create_table_query = '''
CREATE TABLE IF NOT EXISTS predict (
    gu STRING,
    ymd DATE,
    hour INT,
    traffic FLOAT
)
'''
# 테이블 생성 쿼리 실행
cursor.execute(create_table_query)

# 커밋
conn.commit()

# 커넥션 닫기
conn.close()

In [7]:
air_df


,ymd,hour,spot,pm10,pm25,o3,no2,co,so2
0,20231026,20,용산구,49.0,32.0,0.059,0.013,0.5,0.004
1,20231026,20,은평구,62.0,42.0,0.043,0.017,0.7,0.003
2,20231026,20,광진구,44.0,29.0,0.0,0.018,0.6,0.002
3,20231026,20,성동구,46.0,32.0,0.051,0.015,0.4,0.002
4,20231026,20,중랑구,47.0,31.0,0.046,0.015,0.4,0.002
...,...,...,...,...,...,...,...,...,...
270,20231027,20,용산구,25.0,14.0,0.023,0.026,0.7,0.003
271,20231027,20,은평구,22.0,13.0,0.011,0.026,0.5,0.002
272,20231027,20,마포구,24.0,12.0,0.019,0.031,0.5,0.003
273,20231027,20,중랑구,21.0,12.0,0.021,0.024,0.4,0.002


In [8]:
air_df['ymd'] = pd.to_datetime(air_df['ymd'], format='%Y%m%d')

In [9]:
col= { 'spot': 'gu','hour':'time'}
air_df.rename(columns=col,inplace=True)
air_df = air_df[air_df['gu'] !='강북구']

In [10]:
air_df['year']=air_df['ymd'].dt.year
air_df['month']=air_df['ymd'].dt.month
air_df['day'] =air_df['ymd'].dt.day
air_df = air_df.drop(columns='ymd')

/tmp/ipykernel_49/2759513146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  air_df['year']=air_df['ymd'].dt.year
/tmp/ipykernel_49/2759513146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  air_df['month']=air_df['ymd'].dt.month
/tmp/ipykernel_49/2759513146.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [11]:

# 파일에서 객체 로드
label_encoder = joblib.load('label_encoder_classes.joblib')

# 로드된 객체를 사용하여 변환
air_df['gu'] = label_encoder.transform(air_df['gu'])

# 또는 반대로 변환도 가능
# decoded_labels = label_encoder.inverse_transform(encoded_labels)

/usr/local/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
air_df

,time,gu,pm10,pm25,o3,no2,co,so2,year,month,day
0,20,19,49.0,32.0,0.059,0.013,0.5,0.004,2023,10,26
1,20,20,62.0,42.0,0.043,0.017,0.7,0.003,2023,10,26
2,20,4,44.0,29.0,0.0,0.018,0.6,0.002,2023,10,26
3,20,14,46.0,32.0,0.051,0.015,0.4,0.002,2023,10,26
4,20,23,47.0,31.0,0.046,0.015,0.4,0.002,2023,10,26
...,...,...,...,...,...,...,...,...,...,...,...
270,20,19,25.0,14.0,0.023,0.026,0.7,0.003,2023,10,27
271,20,20,22.0,13.0,0.011,0.026,0.5,0.002,2023,10,27
272,20,11,24.0,12.0,0.019,0.031,0.5,0.003,2023,10,27
273,20,23,21.0,12.0,0.021,0.024,0.4,0.002,2023,10,27


In [13]:
air_df = air_df.astype(float)

In [14]:
air_df['time']=air_df['time'].astype(int)
air_df['gu']=air_df['gu'].astype(int)
air_df['month']=air_df['month'].astype(int)
air_df['year']=air_df['year'].astype(int)
air_df['day']=air_df['day'].astype(int)
air_df['gu']= air_df['gu'].astype(int)

In [15]:
air_df.dtypes

time       int64
gu         int64
pm10     float64
pm25     float64
o3       float64
no2      float64
co       float64
so2      float64
year       int64
month      int64
day        int64
dtype: object

In [16]:
model = load('./lgbm_model.pkl')
preds = model.predict(air_df)

[LightGBM] [Warning] min_sum_hessian_in_leaf is set with min_child_weight=0.001, will be overridden by min_hessian=0.05. Current value: min_sum_hessian_in_leaf=0.05
[LightGBM] [Warning] feature_fraction is set with sub_feature=0.345, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.345
[LightGBM] [Warning] min_data_in_leaf is set with min_data=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] Unknown parameter: _n_classes
[LightGBM] [Warning] bagging_fraction is set=0.85, subsample=1.0 will be ignored. Current value: bagging_fraction=0.85
[LightGBM] [Warning] bagging_freq is set=40, subsample_freq=0 will be ignored. Current value: bagging_freq=40


In [17]:
air_df['traffic']=preds

In [18]:
air_df['gu'] = label_encoder.inverse_transform(air_df['gu'])

In [19]:
air_df

,time,gu,pm10,pm25,o3,no2,co,so2,year,month,day,traffic
0,20,용산구,49.0,32.0,0.059,0.013,0.5,0.004,2023,10,26,16099.486645
1,20,은평구,62.0,42.0,0.043,0.017,0.7,0.003,2023,10,26,14563.640705
2,20,광진구,44.0,29.0,0.000,0.018,0.6,0.002,2023,10,26,14261.091937
3,20,성동구,46.0,32.0,0.051,0.015,0.4,0.002,2023,10,26,14999.549175
4,20,중랑구,47.0,31.0,0.046,0.015,0.4,0.002,2023,10,26,14579.656343
...,...,...,...,...,...,...,...,...,...,...,...,...
270,20,용산구,25.0,14.0,0.023,0.026,0.7,0.003,2023,10,27,15726.975639
271,20,은평구,22.0,13.0,0.011,0.026,0.5,0.002,2023,10,27,14328.716953
272,20,마포구,24.0,12.0,0.019,0.031,0.5,0.003,2023,10,27,15280.515495
273,20,중랑구,21.0,12.0,0.021,0.024,0.4,0.002,2023,10,27,14334.609999


In [20]:
gu_df = pd.read_csv('./spot_data_guz.csv')

In [21]:
cols ={ 'spot_num' : 'spot' }
gu_df.rename(columns=cols,inplace=True)

In [22]:
tr_df = pd.merge(gu_df, tr_df, on=['spot'])

In [23]:
tr_df.drop(columns='spot',inplace=True)

In [24]:
tr_df.dtypes

gu      object
ymd     object
hour    object
vol     object
dtype: object

In [25]:
tr_df['vol']=tr_df['vol'].astype(int)
tr_df['gu']=tr_df['gu'].astype(object)
tr_df['hour']=tr_df['hour'].astype(int)
tr_df['ymd']=tr_df['ymd'].astype(int)

In [26]:
tr_df = tr_df.groupby(['gu','ymd','hour'])[['vol']].sum().reset_index()

In [27]:
tr_df

,gu,ymd,hour,vol
0,강남구,20231026,20,24876
1,강남구,20231027,8,118836
2,강남구,20231027,10,27482
3,강남구,20231027,12,27231
4,강남구,20231027,14,27522
...,...,...,...,...
259,중랑구,20231027,18,10829
260,중랑구,20231027,20,9342
261,중랑구,20231030,8,9921
262,중랑구,20231030,10,8635


In [28]:
new_df=pd.merge(tr_df,air_df, on=['gu'])
drop_cols = { 'time','year','month','day'}
new_df.drop(columns=drop_cols,inplace=True)
new_df['ymd']= pd.to_datetime(new_df['ymd'],format='%Y%m%d')
new_df['hour'] = new_df['hour'].astype(int)

In [29]:
use_col = ['gu','ymd','hour','traffic']

In [32]:
l_df = new_df[use_col]

In [36]:
l_df.dtypes


gu                 object
ymd        datetime64[ns]
hour                int64
traffic           float64
dtype: object

In [37]:
l_df

,gu,ymd,hour,traffic
0,강남구,2023-10-26,20,16782.118966
1,강남구,2023-10-26,20,16968.856847
2,강남구,2023-10-26,20,17212.805918
3,강남구,2023-10-26,20,16731.306572
4,강남구,2023-10-26,20,16681.142123
...,...,...,...,...
2899,중랑구,2023-10-30,12,14464.737946
2900,중랑구,2023-10-30,12,14412.108012
2901,중랑구,2023-10-30,12,14407.604779
2902,중랑구,2023-10-30,12,14419.941426
